# Spark

## 상태
 
* 2016-07-11월
    * [ok] prebuilt 설치 - spark1.6 for hadoop2.0
    * [ok] ipython notebook -> findspark를 사용 (아래 방법 1)
    * [mac ok] ipython notebook kernel (아래 방법 2)
    * [ok] IPYTON_OPTS (아래 방법 3)


## 설치

### prerequisites

* jdk
* scala
* python 2.7
* py4j (선택적, pip install py4j하면 PYTHONPATH에 py4j가 불필요)

### 설치 방법

* 방법1: prebuilt distribution
    * HDFS를 사용하려면, 하둡버전을 정해서 설치파일을 내려받고 압축을 풀어준다
    (Hadoop없는 설치하면 오류. 아래 참조)
    * 다운로드하고 압축을 풀어준다 "https://spark.apache.org/downloads.html"   
    ```
    $ tar xvfz spark-1.x.x-bin-hadoop2.x.tgz
    ```

* 방법2: source code를 github에서 내려받아서 컴파일

* 방법3: apt로 설치하는 경우
    * spark 실행에 필요한 shell을 설치하지 않는 듯하다. 위 tar를 다운로드 설치하는 것으로 한다.
    * when install from apt (available as of Ubuntu 16.04)
        ```
        sudo apt-get install scala
        sudo apt-get install spark
        ```

### 설치 확인

* 방법1로 spark-shell, pyspark를 실행하는데 문제가 없다.

    * 경로설정을 해주지 않았으면,
    ```
    $ cd ~/Downloads/spark-1.6.0-bin-hadoop2.6/bin
    $ spark-shell
    ```

    * 경로설정을 하는 경우
    ```
    export SPARK_HOME=$HOME/Downloads/spark-1.6.0-bin-hadoop2.6
    export PATH=$SPARK_HOME/bin:$PATH
    ```
    
    * 환경변수 설정 (bashrc)
    ```
    export SPARK_HOME="$HOME/Downloads/spark-1.6.0-bin-hadoop2.6"
    export PYSPARK_SUBMIT_ARGS="--packages graphframes:graphframes:0.1.0-spark1.6 --master spark://127.0.0.1:7077 pyspark-shell"
    ```


In [1]:
!java -version

openjdk version "1.8.0_91"
OpenJDK Runtime Environment (build 1.8.0_91-8u91-b14-3ubuntu1~16.04.1-b14)
OpenJDK 64-Bit Server VM (build 25.91-b14, mixed mode)


In [2]:
!scala -version

Scala code runner version 2.11.6 -- Copyright 2002-2013, LAMP/EPFL


## 오류

* SparkContext: Error initializing SparkContext
    * /etc/hostname이 지금 설정과 다른 경우의 오류
    * 변경하려면
        ```
        $ scutil --set HostName jsl.com
        ```
        
    * 확인하려면   
        ```
        $ hostname
        ```

* cannot import name accumulators
    * bin/pyspark 명령어를 보면  path 설정 나옴
    * 일단 시작하면 path추가 안되는 모양???
    * findspark를 설치??
        * https://github.com/minrk/findspark/blob/master/findspark.py

* 하둡 아닌 것을 내려받아 설치한 경우
    * download and tar to ~/Downloads/spark-x.x.x/
    * when running ~/Downloads/spark-1.6.1/spark-shell
        * error Failed to find Spark assembly spark
            * so need to install sbt
            ```
            download tar from http://www.scala-sbt.org/
            tar -xvf sbt.tar
            ```
            
            * this problem went away with the one prebuilt with hadoop


* 실행 오류
    * java.net.BindException: Address already in use Address already in use
    ```
    ps -aux | grep spark
    kill -9 3370
    ```

## IPython shell을 사용하려면

* pyspark는 python shell을 사용한다. ipython shell을 실행하려면:

* 방법 1: 단순한 방식
```
 $ export PYSPARK_DRIVER_PYTHON=ipython
 $ ./bin/pyspark 이러면 ipython를 shell로 사용하게 된다. 
 또는
 $ PYSPARK_PYTHON=python2.7 ./bin/pyspark
```

* 방법2: profile로 환경 설정하기
    * ipython profile 생성. 아래 명령을 실행하면, 생성된 파일을 수정한다.
        ```
        ipython profile create pyspark
        ```

    * profile startup 파일 생성
        ```
        vim ~/.ipython/profile_pyspark/startup/00-pyspark-setup.py 
        ```

        ```
        import os
        import sys

        if 'SPARK_HOME' not in os.environ:
            os.environ['SPARK_HOME'] = '/home/jsl/Downloads/spark-1.6.0-bin-hadoop2.6'
        SPARK_HOME = os.environ.get('SPARK_HOME', None)
        sys.path.insert(0, SPARK_HOME + "/python")

        sys.path.insert(0, os.path.join(SPARK_HOME, "python/lib/py4j-0.9-src.zip")
        execfile(os.path.join(SPARK_HOME, 'python/pyspark/shell.py'))
        ```

    * ipython shell 실행
        ```
        $ ipython console --profile=pyspark
        ```


## IPython 노트북에서 실행하려면

* 방법1: 그냥 ipython notebook을 실행하고, 셀에 다음 내용을 입력한다.
    ```
    import os
    import sys 
    os.environ["SPARK_HOME"]='/Users/media/Downloads/spark-2.1.0-bin-hadoop2.7'
    os.environ["PYLIB"]=os.environ["SPARK_HOME"]+'/python/lib'
    sys.path.insert(0,os.environ["PYLIB"]+'/py4j-0.10.4-src.zip')
    sys.path.insert(0,os.environ["PYLIB"]+'/pyspark.zip')
    ```

    * 그리고 sc를 만들어 주면 된다.
    ```
    import pyspark
    conf=pyspark.SparkConf()
    conf = pyspark.SparkConf().setAppName("myAppName")
    sc = pyspark.SparkContext(conf=conf)
    sc.version
    ```

* 방법2: findspark 라이브러리를 설치

In [5]:
import os
home=os.getenv("HOME")

In [6]:
import findspark
findspark.init(os.path.join(home,"Downloads/spark-1.6.0-bin-hadoop2.6"))

In [7]:
import pyspark
sc = pyspark.SparkContext(appName="myAppName")

In [8]:
print sc

* 방법3: ipython notebook kernel을 설정 (mac ok 20160902 fri)

    * [안해도ok package사용] lib에 graphframes.jar 다운로드 저장. package 사용.
    * [안해도ok] 설정변경하려면, 'conf/spark-env.sh'을 변경하고 "PYSPARK_SUBMIT_ARGS"에 추가 --master spark://127.0.0.1:7077
    * [안해도ok] pip install py4j하면 PYTHONPATH에 py4j가 불필요
    * kernel.json 작성 - 작성 후, ipython notebook을 실행하면 kernel에 pySpark가 올라옴.
    ```        
    $ mkdir -p ~/.ipython/kernels/pyspark
    $ vim ~/.ipython/kernels/pyspark/kernel.json
    ```

    * kernel.json은 아래 내용을 수정해서 저장
```
{
    "display_name": "pySpark (Spark 1.6.0) with graphFrames",
    "language": "python",
    "argv": [
        "/usr/bin/python2.7",
        "-m",
        "ipykernel",
        "-f",
        "{connection_file}"
    ],  
    "env": {
        "SPARK_HOME": "/Users/media/Downloads/spark-1.6.0-bin-hadoop2.6",
        "PYTHONPATH": "/Users/media/Downloads/spark-1.6.0-bin-hadoop2.6/python:/Users/media/Downloads/spark-1.6.0-bin-hadoop2.6/python/lib/py4j-0.9-src.zip",
        "PYTHONSTARTUP": "/Users/media/Downloads/spark-1.6.0-bin-hadoop2.6/python/pyspark/shell.py",
        "PYSPARK_SUBMIT_ARGS": "--packages graphframes:graphframes:0.1.0-spark1.6 --master spark://127.0.0.1:7077 pyspark-shell",
        "SPARK_DRIVER_MEMORY":"10G"
     }   
}
```

* 방법4
    * 명령창에서 다음 명령어
        ```
        $ IPYTHON_OPTS="notebook" ~/Downloads/spark-1.6.0-bin-hadoop2.6/bin/pyspark
        ```
        
        또는
        ```
        $ IPYTHON_OPTS="notebook --pylab inline" ~/Downloads/spark-1.6.0-bin-hadoop2.6/bin/pyspark
        ```
        
    * 그리고 브라우저에서
    ```
    import pyspark
    sc.stop() # Cannot run multiple SparkContexts at once
    sc = pyspark.SparkContext(appName="myAppName")
    print sc
    ```
    * 다음 명령어는 오류 -> 7077 can not be reached
    ```
    MASTER="spark://127.0.0.1:7077" SPARK_EXECUTOR_MEMORY="6G" IPYTHON_OPTS="notebook" ~/Downloads/spark-1.6.0-bin-hadoop2.6/bin/pyspark
    ```

## jar를 추가

* Scala, Java보다 python에서 추가하는 것은 쉽지 않다.
* Add jar to pyspark when using notebook
http://stackoverflow.com/questions/31677345/add-jar-to-pyspark-when-using-notebook?rq=1

### Spark package 검색

```
https://spark-packages.org/
```

### 방법

* [ok] pyspark --jars
    ```
    $ bin/pyspark --py-files lib/graphframes.jar --jars lib/graphframes.jar 
    Welcome to
          ____              __
         / __/__  ___ _____/ /__
        _\ \/ _ \/ _ `/ __/  '_/
       /__ / .__/\_,_/_/ /_/\_\   version 1.6.0
          /_/

    Using Python version 2.7.12 (default, Jul  1 2016 15:12:24)
    SparkContext available as sc, HiveContext available as sqlContext.
    >>> from graphframes import *
    ```

* packages 
    * [ok] spark-submit --packages
        ```
        ./bin/spark-submit src/ds_spark_dataframe.py \
           --packages graphframes:graphframes:0.1.0-spark1.6
        ```

    * [ok] spark-defaults.conf

config file | 설명
----------|----------
spark-default.conf | system properties
spark-env.conf | 

        * 예:
        ```
        spark.jars.packages=graphframes:graphframes:0.1.0-spark1.6
        ```

        * 또는 import jars to both driver AND executor. 와일드카드 '*'도 가능함
        ```
        spark.driver.extraClassPath /path/to/my.jar
        spark.executor.extraClassPath /path/to/my.jar
        ```
        
    * [ok] SparkConf (spark-defaults.conf에 packages 추가한 후)
        ```
        conf=pyspark.SparkConf()
        conf = pyspark.SparkConf().setAppName("myAppName")
        conf.set("spark.mongodb.input.uri","mongodb://127.0.0.1/test.myCollection2")
        conf.set("spark.mongodb.output.uri","mongodb://127.0.0.1/test.myCollection2")
        sc = pyspark.SparkContext(conf=conf)
        ```

* [not yet] addJar()
    ```
    SparkContext.addJar(...) method
    SparkContext.addFile(...) method
    ```
    
* [not yet] loading jars
    ```
    from py4j.java_gateway import java_import
    ```

* [nok] os.environ - submit_args
    * IPyton Notebook에서 잘 안되는 듯 class-not-found errors. python kernel이 일단 실행되고 나면, 동적으로 추가하는 것이 안되는 듯. (subprocess.Popen은 env값을 가져올 수 있지 않나??)
    ```
    import os
    os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages graphframes:graphframes:0.2.0-spark2.0-s_2.11'
    ```
* SPARK_CLASSPATH - spark-env.sh